<a href="https://colab.research.google.com/github/helenmck1/SDS-bootcamp-2023_Tokyo/blob/main/christopher_session/SDS_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting AirBnB Prices using Carto open data and Machine Learning

<img src="https://github.com/helenmck1/SDS-bootcamp-2023_Tokyo/blob/main/christopher_session/images/warning.png?raw=1"  width="600" height="250">

<img src="https://github.com/helenmck1/SDS-bootcamp-2023_Tokyo/blob/main/christopher_session/images/gpu_setting.png?raw=1"  width="400" height="350">

![alt text for screen readers](images/risk_image.png "Predicted AirBnB prices on the left, actual prices on the right.")

# Libraries / ライブラリー

In [ ]:
# Install packages
# パッケージのインストール
!pip install pydeck-carto geopandas carto-auth[carto-dw] db_dtypes -q xgboost category-encoders numpy shap matplotlib seaborn

In [ ]:
# Carto / GoogleBigQuery
import pydeck as pdk
import pydeck_carto as pdkc
from carto_auth import CartoAuth
from google.cloud import bigquery

# Plotting / プロット
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Analysis / 分析
import math
import numpy as np
import pandas as pd
import geopandas as gpd

# Machine Learning / 機械学習
import xgboost as xgb
import category_encoders as ce
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    LeaveOneOut,
    cross_val_score,
    train_test_split,
)

# Authentication / 認証

In [ ]:
# Authenticate Carto
# Cartoを認証する
carto_auth = CartoAuth.from_oauth()
carto_dw_client = carto_auth.get_carto_dw_client()

In [ ]:
# Determine the user's dataset IDs
# ユーザーのデータセットIDを決定する
dataset_id = 'shared_us'
tables = carto_dw_client.list_tables(dataset_id)
carto_user_id = list(tables)[0].project.split('-')[-1]

cdw_dataset = f"carto-dw-ac-{carto_user_id}"
cdo_dataset =  f"carto-data.ac_{carto_user_id}"

print(f"Carto Data Warehouse Dataset: {cdw_dataset}")
print(f"Carto Data Warehouse Dataset: {cdo_dataset}")

# Accessing Data / データへのアクセス

In [ ]:
# List datasets in Carto Data Warehouse
# Cartoデータウェアハウスのデータセットの表示
datasets = list(carto_dw_client.list_datasets())

if datasets:
    print("Datasets in CARTO Data Warehouse:")
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("CARTO Data Warehouse project does not contain any datasets.")

In [ ]:
# List tables in chosen dataset
# 選択したデータセットのテーブルをリストアップする
dataset_id = 'shared_us'
tables = carto_dw_client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
# Query the London Listings table and visualize as GeoDataFrame
# London Listingsテーブルを照会し、GeoDataFrameとして可視化する。
table = carto_dw_client.get_table(f"{cdw_dataset}.shared_us.london_listings_2023_joined")
gdf = carto_dw_client.list_rows(table).to_geodataframe(
    geography_column="geom", create_bqstorage_client=False
)
display(gdf.head())

# Visualizing Data / データの可視化

## CARTO Data Warehouse / CARTOデータウェアハウス

In [ ]:
# Register CartoLayer in pydeck
# pydeckにCartoLayerを登録する。
pdkc.register_carto_layer()

# Render first CartoLayer CartoLayer
# 最初の CartoLayer CartoLayer をレンダリングする。
layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT geom, price FROM `{cdw_dataset}.shared_us.london_listings_2023_joined`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),
    point_radius_min_pixels=2.5,
    get_fill_color=pdkc.styles.color_bins(
        "price", [0, 100, 200, 300, 400, 500], "PinkYl"
    ),
    get_line_color=[0, 0, 0, 100],
)

# Set initial viewing location/zoom/angle
# 初期表示位置/ズーム/アングルを設定する
view_state = pdk.ViewState(
    latitude=51.50071697877869,
    longitude=-0.12461158468895285,
    zoom=8
)

# Initialize map
# マップの初期化
pdk.Deck(layer, map_style=pdk.map_styles.ROAD, initial_view_state=view_state).to_html(
    iframe_height=800
)

# Data preparation / データ準備

## Data cleaning / データクリーニング

In [ ]:
# Load joined Carto Workflow table into a DataFrame
# 結合されたCarto WorkflowテーブルをDataFrameにロードする。
gdf_load = gdf.copy()

# Create an index on the table
# テーブルにインデックスを作成する
idx = "index"
gdf_load = gdf_load.reset_index()
gdf_load = gdf_load.set_index(idx)
gdf_load

# Drop geometry columns
# ジオメトリの列を削除する
gdf_clean = gdf_load.drop(['h3', 'geom'], axis=1)

# Rename columns created by join in workflow
# ワークフローで結合によって作成された列の名前を変更する
gdf_clean.rename(columns={"population_joined": "population",
                          "leisure_joined": "lesiure",
                          "tourism_joined": "tourism",
                          "transportation_joined": "transportation",
                          "urbanity_joined": "urbanity",
                          "female_joined": "female",
                          "male_joined": "male",
                          "elevation_joined": "elevation"}, inplace=True)
gdf_clean.head()

In [ ]:
# Check data types for each column
# 各列のデータ型をチェックする
gdf_clean.dtypes

In [ ]:
# Force data types for our variables
# 変数のデータ型を強制する
for col in [
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'availability_365',
    'number_of_reviews_ltm',
    'population',
    'female',
    'male',
    'lesiure',
    'tourism',
    'transportation',
    'elevation',
    'mobility'
]:
  gdf_clean[col] = gdf_clean[col].astype('float64')

gdf_clean.head()

## Null values / ヌル値

In [ ]:
gdf_selected = gdf_clean.copy()
# Display null values
# ヌル値を表示する
display(gdf_selected.isnull().sum())

# Fill remaining null values with 0
# 残りのヌル値を0で埋める
gdf_selected = gdf_selected.fillna(0)
display(gdf_selected.isnull().sum())

## Outliers / アウトライアーズ

In [ ]:
# Use df.describe to check percentiles for each numerical colummn
# df.describeを使って各数値列のパーセンタイルをチェックする
gdf_selected.describe(percentiles=[.25, .5, .75, .95, .99])

In [ ]:
# Remove data from selected columns not within the 99th percentile
# 選択された列から99パーセンタイル以内のデータを削除する。
gdf_selected = gdf_selected[gdf_selected['price'] >= 1]
gdf_selected = gdf_selected[gdf_selected['price'] <= 1570.000000]
gdf_selected = gdf_selected[gdf_selected['minimum_nights'] <= 90]
gdf_selected = gdf_selected[gdf_selected['number_of_reviews'] <= 209.000000]
gdf_selected = gdf_selected[gdf_selected['number_of_reviews_ltm'] <= 61.0]
gdf_selected

## Choosing Variables / 変数の選択

**CATEGORICAL VARIABLES** / **カテゴリー変数** \
neighbourhood　エリア区分\
room_type　部屋のタイプ\
urbanity　都市の度合\
work_zone　職場ゾーン\
work_zone_specific　働く人のタイプによるエリア分け\
\
**NUMERICAL VARIABLES** / **数値変数** \
minimum_nights　最低宿泊日数\
number_of_reviews　レビュー数\
availability_365　年間稼働率\
number_of_reviews_ltm　直近10ヶ月のレビュー数\
reviews_per_month　月次レビュー数\
transportation　交通関連POIの数\
tourism　観光関連POI数\
lesiure　レジャー関連POIの数\
population　人口\
male　男性人口\
female　女性人口\
mobility　世帯の変動\
\
**TARGET VARIABLE** / **目的変数**\
price　価格

In [ ]:
""" カテゴリー変数
neighbourhood
room_type
urbanity
work_zone
work_zone_specific
"""

"""数値変数
minimum_nights
number_of_reviews
availability_365
number_of_reviews_ltm
reviews_per_month
transportation
tourism
lesiure
population
male
female
mobility
"""

In [ ]:
categorical_variables = ['neighbourhood','urbanity']
numerical_variables = ['']
target = "price"

In [ ]:
# Declare numerical, categorical and target variables
# 数値変数、カテゴリー変数、ターゲット変数の宣言
categorical_variables = [
    "neighbourhood",
    "room_type",
    "urbanity",
    "work_zone_specific"
]

numerical_variables = [
    "minimum_nights",
    "number_of_reviews",
    "reviews_per_month",
    "availability_365",
    "population",
    "lesiure",
    "tourism",
    "transportation",
    "mobility",
]

target = "price"

## Feature Analysis / 特徴分析

In [ ]:
# Count plot for target variable
# ターゲット変数のカウントプロット
sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [10, 10]})
sns.displot(gdf_selected['price'], kde=False, bins=20,).set(xlabel='Price', ylabel='Count')

In [ ]:
# Count plot for target numerical variables
# 数値変数のカウントプロット
num_len = len(numerical_variables)
cols = 5
row_fun = lambda len, cols: math.floor(len / cols) + 1 if len % cols != 0 else math.floor(len / cols)
rows = row_fun(num_len, cols)
gdf_selected[numerical_variables].hist(bins=15, figsize=(15, 6), layout=(rows, 5))

In [ ]:
# Count plot for target categorical variables
# カテゴリー変数のカウントプロット

num_len = len(categorical_variables)
cols = 2
row_fun = lambda len, cols: math.floor(len / cols) + 1 if len % cols != 0 else math.floor(len / cols)
rows = row_fun(num_len, cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
axes = axes.ravel()
for col, ax in zip(categorical_variables, axes):
    sns.histplot(gdf_selected[col], ax=ax).set_xticklabels(
        ax.get_xticklabels(), rotation=90
    )

fig.tight_layout()
plt.show()

## Encoding / エンコーディング

![alt text for screen readers](images/encoding1.png "One Hot Encoding.")

![alt text for screen readers](images/encoding2.png "Label Encoding.")

In [ ]:
gdf_encoded = gdf_selected[categorical_variables + numerical_variables + [target]].copy()
# Display categorical features and their respective cardinality
# カテゴリカルな特徴とそれぞれのカーディナリティを表示します

high_card = []
low_card = []
for cat in categorical_variables:
    if gdf_encoded[cat].nunique() >= 15:
       high_card.append(cat)
    else:
        low_card.append(cat)

# Nominal variable and cardinality < 15, use OneHot Encoding
# 名目変数でカーディナリティが15未満の場合、OneHotエンコーディングを使用する。
gdf_encoded = pd.get_dummies(gdf_encoded, columns=low_card, dtype=int)

# Nominal variable and cardinality >= 15, use Target Encoding
# 名目変数でカーディナリティが15を超える場合は、ターゲットエンコーディングを使用する。
encoder = ce.TargetEncoder()
for hc in high_card:
    gdf_encoded[hc] = encoder.fit_transform(gdf_encoded[hc], gdf_encoded[target])
gdf_encoded

In [ ]:
# Split dataset into dependent and independent variables (X and y)
# データセットを従属変数と独立変数 (X と y) に分割する
X = gdf_encoded.drop("price", axis=1).copy()
y = gdf_encoded["price"].copy()

# Split X and y data into train and test data 80/20
# XとYのデータを訓練データとテストデータに分ける 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=.2)

# Models / モデル

## XGBOOST

### Default Model / デフォルトモデル

In [ ]:
# Create model instance
# モデルのインスタンスを作成する
xgb_model = xgb.XGBRegressor(
    seed=39,
    eval_metric=["mae", "rmse"]
)

### Optimized Model / 最適化モデル

In [ ]:
# Create model instance
# モデルのインスタンスを作成する
optimized_parameters = {
    'n_estimators': 100,
    'colsample_bytree': 0.5,
    'gamma': 0.1,
    'learning_rate': 0.05,
    'max_depth': 12,
    'min_child_weight': 1
}
xgb_model = xgb.XGBRegressor(
    **optimized_parameters,
    seed=39,
    eval_metric=["mae", "rmse"],
    tree_method='gpu_hist',
    device="cuda",
    predictor = 'gpu_predictor'
)

In [ ]:
# Train the model
# モデルをトレーニングする
eval_set = [(X_train, y_train), (X_test, y_test)]
xgb_model.fit(X_train, y_train, verbose=0, eval_set=eval_set)

In [ ]:
# Make predictions on our test data
# テストデータで予測を行う
preds = xgb_model.predict(X_test)
predictions = [round(value) for value in preds]
print("Predicted prices:", predictions[0:5])
print("Actual prices:   ", [round(value) for value in list(y_test)][0:5])

In [ ]:
# Evaluate model
# モデルを評価する

# Higher is better
# R2 score / R2スコア
print("R^2 :", r2_score(y_test, predictions))
# Adjusted R2 / 調整後R2
print("Adjusted R^2 :", 1 - (1-xgb_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1))
print()
# Less is better
# Mean Average Error / 平均平均誤差
print("MAE :", mean_absolute_error(y_test,predictions))
# Root Mean Squared Error / 平均二乗誤差
print("RMSE:", np.sqrt(mean_squared_error(y_test, predictions)))

## Validation & Plotting / バリデーションとプロッティング

In [ ]:
# Calculate the shap values (GPU reccomended)
# シャップ値を計算する（GPU推奨）
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)
expected_value = explainer.expected_value

if isinstance(expected_value, list):
    expected_value = expected_value[1]
print(f"Explainer expected value: {expected_value}")

In [ ]:
# Create summary plot, showing the most impactful features
# 最もインパクトのある特徴を示すサマリープロットを作成する
shap.summary_plot(shap_values, X_test, plot_type="bar", show=True)

In [ ]:
# Create force plot for the same predictions, a different way of visualizing feature impact
# 同じ予測に対してフォースプロットを作成し、特徴の影響を別の方法で視覚化する
features = X_test.iloc[0:1]
shap_values = explainer.shap_values(features)

shap.force_plot(expected_value, shap_values, features, text_rotation=10, matplotlib=True)

# Prediction / 予想

In [ ]:
# Make predictions and joined them back to test data
# 予測を立て、それをテストデータに結びつける
gdf_out = X_test.copy()
gdf_out["price_predicted"] = xgb_model.predict(X_test)
gdf_out = gdf_out.merge(gdf_load, left_on="index", right_on="index")[
    ["h3", "price", "price_predicted"]
]
gdf_out.reset_index(drop=True, inplace=True)

# Upload result to our Carto account and overwrite same table if already present
# 結果をCartoアカウントにアップロードし、同じテーブルが存在する場合は上書きする
job_config = bigquery.LoadJobConfig(schema=[], write_disposition="WRITE_TRUNCATE")
carto_dw_client.load_table_from_dataframe(
    gdf_out,
    f"{cdw_dataset}.shared_us.london_listings_2023_predicted",
    job_config=job_config,
).result()

In [ ]:
# List tables again and see if out new table is there
# テーブルを再度リストアップし、新しいテーブルがあるかどうかを確認する
dataset_id = 'shared_us'
tables = carto_dw_client.list_tables(dataset_id)

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
# Create final map to visualize our predictions
# 予測を視覚化する最終マップを作成する
pdkc.register_carto_layer()

layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT h3, price_predicted, price FROM `{cdw_dataset}.shared_us.london_listings_2023_predicted`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),

    aggregation_exp=pdk.types.String("avg(price_predicted) as price_predicted"),
    aggregation_res_level=8,
    geo_column=pdk.types.String("h3"),

    get_fill_color=pdkc.styles.color_bins("price_predicted",[0, 100, 200, 300, 400, 500], "Sunset"),
    opacity=0.4,


    get_elevation='properties.price_predicted',
    elevation_scale=10,

    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    auto_highlight=True,
    pickable=True
)

tooltip = {"html": "<b>H3 Index:</b> {id} <br /><b>Predicted Price:</b> {price_predicted}"}

view_state = pdk.ViewState(
    latitude=51.50071697877869,
    longitude=-0.12461158468895285,
    zoom=8,
    pitch=45,
    bearing=0
  )

pdk.Deck(layer, map_style=pdk.map_styles.DARK, tooltip=tooltip, initial_view_state=view_state).to_html(iframe_height=800)

In [ ]:
# Create final map to visualize our predictions
# 予測を視覚化する最終マップを作成する
pdkc.register_carto_layer()

layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT h3, price FROM `{cdw_dataset}.shared_us.london_listings_2023_predicted`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),

    aggregation_exp=pdk.types.String("avg(price) as price"),
    aggregation_res_level=8,
    geo_column=pdk.types.String("h3"),

    get_fill_color=pdkc.styles.color_bins("price",[0, 100, 200, 300, 400, 500], "Sunset"),
    opacity=0.4,

    get_elevation='properties.price',
    elevation_scale=10,

    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    auto_highlight=True,
    pickable=True
)

tooltip = {"html": "<b>H3 Index:</b> {id} <br /><b>Actual Price:</b> {price}"}

view_state = pdk.ViewState(
    latitude=51.50071697877869,
    longitude=-0.12461158468895285,
    zoom=8,
    pitch=45,
    bearing=0
  )

pdk.Deck(layer, map_style=pdk.map_styles.DARK, tooltip=tooltip, initial_view_state=view_state).to_html(iframe_height=800)